In [ ]:
from selenium import webdriver
import time
from datetime import datetime
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException
import pandas as pd
import calendar
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import requests
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
import os
from selenium.webdriver.chrome.options import Options
import re
import threading
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
# GU LIBRARY
chrome_options = Options()
#chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--enable-precise-memory-info")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 25)
driver.get("https://www.library.georgetown.edu/")

wait = WebDriverWait(driver, 25)
# Store the ID of the original window
original_window = driver.current_window_handle
# Check we don't have other windows open already
assert len(driver.window_handles) == 1

driver.implicitly_wait(5)
search_input = driver.find_element(by=By.ID, value="search-query")
#search_input = driver.find_element_by_id("search-query")
search_input.send_keys("new york times")

driver.implicitly_wait(5)

search_button = driver.find_element(by=By.ID, value="banner-search-button")
search_button.click()
driver.find_element(By.LINK_TEXT, "New York Times Online Access for Research").click()

#nyt_button = driver.find_element_by_link_text("New York Times Online Access for Research")
#nyt_button.click()

# NEW WINDOW OPEN
driver.implicitly_wait(19)
wait.until(EC.number_of_windows_to_be(2))
# Loop through until we find a new window handle
for window_handle in driver.window_handles:
    if window_handle != original_window:
        driver.switch_to.window(window_handle)
        break

# Wait for the new tab to finish loading content
#wait.until(EC.title_is("The New York Times. - Georgetown University"))
driver.implicitly_wait(50)
proquest_nyt = driver.find_element(By.LINK_TEXT, "ProQuest Historical Newspapers: The New York Times with Index")
original_window1 = driver.current_window_handle
proquest_nyt.click()

# Loop through until we find a new window handle
for window_handle in driver.window_handles:
    if window_handle != original_window1 and window_handle != original_window:
        driver.switch_to.window(window_handle)
        break

driver.implicitly_wait(50)
#wait.until(EC.title_is("Single Signon - Georgetown University"))
#login to your Institutional SSN 
username_input = driver.find_element(by=By.ID, value="username")
username_input.send_keys("nm993")
password_input = driver.find_element(by=By.ID, value="password")
password_input.send_keys("")
driver.implicitly_wait(5)

sign_button = driver.find_element(By.CLASS_NAME, 'form-button')
sign_button.click()
driver.implicitly_wait(15)
element = WebDriverWait(driver, 36000).until(
    EC.visibility_of_element_located((By.ID, "trust-browser-button")))
element.click()

In [ ]:
def close_my_research_account_popup(driver):
    while True:
        try:
            # Check for the popup every second
            time.sleep(1)
            popup = driver.find_element(By.ID, "myModalLabel_getSessionInterruptPrompt")
            if popup:
                close_button = driver.find_element(By.XPATH, "//button[@aria-label='Close Create a My Research account? dialog']")
                close_button.click()
                print("My Research account popup closed.")
        except NoSuchElementException:
            # Popup not found, continue checking
            continue
        except Exception as e:
            print(f"Unexpected error while checking for popup: {e}")
            break


def handle_problem_popup(driver):
    while True:
        try:
            # Check for the popup every second
            time.sleep(1)
            popup_header = driver.find_element(By.XPATH, "//div[@class='header_border_bottom']/h1[text()='We seem to have encountered a problem.']")
            if popup_header:
                close_button = driver.find_element(By.XPATH, "//button[@class='pull-right btn btn-primary' and @type='button']")
                close_button.click()
                print("Problem encountered popup closed.")
        except NoSuchElementException:
            # Popup not found, continue checking
            continue
        except Exception as e:
            print(f"Unexpected error while checking for problem popup: {e}")
            break

# Start the popup handler threads
research_account_thread = threading.Thread(target=close_my_research_account_popup(driver), args=(driver,))
problem_popup_thread = threading.Thread(target=handle_problem_popup, args=(driver,))

research_account_thread.start()
problem_popup_thread.start()


In [ ]:
# THIS IS FROM WINDOWS


# CHANGE OUTPUT DIRECTORY DEPENDING ON YOUR PC
output_directory = os.path.expanduser(f'~/Desktop/Lamis/titles/')

def check_document_unavailable():
    try:
        #Check if the "Document Unavailable" text is present on the page
        WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Document Unavailable')]"))
        )
        # WebDriverWait(driver, 1).until(
        #     EC.presence_of_element_located((By.XPATH, "//h1[text()='Document Unavailable']"))
        # )
        return True
    except TimeoutException:
        return False

def close_research_account_popup():

    try:
        WebDriverWait(driver, 0.5).until(
            EC.presence_of_element_located((By.ID, 'myModalLabel_getSessionInterruptPrompt'))
        )
        close_button = driver.find_element(By.XPATH, "//button[@data-dismiss='modal' and @aria-label='Close Create a My Research account? dialog']")
        close_button.click()
        print("Closed 'Create a My Research account' popup.")
    except:
        pass  # Popup did not appear

def navigate_back_to_issue():
    try:
        back_to_issue_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, "Back to issue"))
        )
        back_to_issue_button.click()
        #print("Navigated back to issue using 'Back to issue' button")
    except TimeoutException:
        print("Could not find 'Back to issue' button, using driver.back() instead")
        for _ in range(3):  # Retry up to 3 times if the element is not found
            try:
                wait.until(EC.presence_of_element_located((By.ID, 'issueSelected')))
                return True

            except:
                print("Element 'issueSelected' not found, navigating back once more...")
                driver.back()
                time.sleep(1)

def navigate_back_and_wait():

    for _ in range(4):
        driver.back()
    #time.sleep(1)  # Short sleep to allow the page to start loading
    # Wait until the 'issueSelected' element is present
    for _ in range(3):  # Retry up to 3 times if the element is not found
        try:
            wait.until(EC.presence_of_element_located((By.ID, 'issueSelected')))
            return True

        except:
            print("Element 'issueSelected' not found, navigating back once more...")
            driver.back()
            time.sleep(1)
    return False

def wait_after_popup():
    for _ in range(3):  # Retry up to 3 times if the element is not found
        try:
            wait.until(EC.presence_of_element_located((By.ID, 'issueSelected')))
            break
        except:
            print("Element 'issueSelected' not found, navigating back once more...")
            driver.back()
            time.sleep(1)

def select_dropdown_option(select_element_id, value):
    """Fetch the dropdown each time before selecting to avoid staleness."""
    WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.ID, select_element_id))
    )
    element = driver.find_element(By.ID, select_element_id)
    select = Select(element)
    select.select_by_value(value)
    WebDriverWait(driver, 20).until(
        lambda driver: Select(driver.find_element(By.ID, select_element_id)).first_selected_option.get_attribute('value') == value
    )


def is_captcha_present():
    try:
        driver.find_element(By.ID, 'verifyCaptcha')
        return True
    except:
        return False

def wait_for_user_to_solve_captcha():
    print("CAPTCHA detected. Please solve the CAPTCHA in the browser and press Enter here to continue...")
    input("Press Enter to continue after solving the CAPTCHA...")


def get_pdf_url():
    for attempt in range(3):  # Retry mechanism
        try:
            pdf_element = wait.until(EC.presence_of_element_located((By.ID, "embedded-pdf")))
            return pdf_element.get_attribute("src")
        except StaleElementReferenceException:
            if attempt < 2:
                print("Retrying to locate pdf_element due to StaleElementReferenceException...")
                time.sleep(2)
            else:
                raise

# Put your dates here
decade_value = "1970 - 1979"
select_dropdown_option('decadeSelected', str(decade_value))
time.sleep(5)
for year_value in range(1971, 1979):

    select_dropdown_option('yearSelected', str(year_value))
    print(f"Year selected: {year_value}")
    time.sleep(3)
    for month_value in range(1, 13):

        # CREATE DATAFRAME
        column_names = ["Date"]
        column_names1 = [f"Title {i+1}" for i in range(100)]
        column_names2 = column_names + column_names1
        df = pd.DataFrame(columns=column_names2)

        select_dropdown_option('monthSelected', f"{month_value:02}")
        print(f"Month selected: {month_value:02}")
        time.sleep(3)
        days = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'issueSelected'))
        )
        time.sleep(3)
        day_select = Select(days)
        day_data = [option.get_attribute('value') for option in day_select.options if '#' in option.get_attribute('value')]

        if day_data:
            print(f"Day data for {year_value}-{month_value:02}: {day_data}")  # Debugging day data
        else:
            print(f"No valid day options found for {year_value}-{month_value:02}")

        for value in day_data:
            day_date, day_text = value.split('#')
            expected_ym = f"0{year_value}Y{month_value:02}Y"
            if day_date.startswith(expected_ym):
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.ID, 'issueSelected'))
                )
                select_dropdown_option('issueSelected', value)
                print(f"Day selected: {day_text}")  # Show which day was selected

                # Click the search button
                WebDriverWait(driver, 1000).until(EC.presence_of_element_located((By.NAME, "searchPubIssues")))
                element = driver.find_element(By.XPATH, "//input[@title='View available contents for the currently selected issue']")
                element.click()

                # Extract the day part from the 'day_text'
                day_of_month = day_text.split(" ")[1].rstrip(",")

                # Convert month number to month name abbreviation
                month_name = calendar.month_abbr[month_value]

                # Format the expected date string
                expected_date = f"Issue contents: {month_name} {day_of_month}, {year_value}"
                #print(f"Expecting date on webpage: {expected_date}")
                # Wait for the issue contents to load with the right date

                #FIX tHIS TO PRESS SEARCH BUTTON AGAIN AND AGAIN
                issue_details_pattern = re.compile(r"Issue contents(?: \(latest available\))?: " + re.escape(expected_date))
                WebDriverWait(driver, 2000).until(
                    EC.text_to_be_present_in_element((By.ID, 'IssueDetails'), expected_date)
                )
                print(f"Loaded content for {expected_date}")

                parsed_date = datetime.strptime(f"{day_of_month} {month_name} {year_value}", "%d %b %Y")
                formatted_date = parsed_date.strftime("%Y-%m-%d")
                # Process the page content (e.g., scraping data)
                page_source = driver.page_source
                soup = BeautifulSoup(page_source, "html.parser")
                titles_with_a1 = []
                for li in soup.find_all("li", class_="resultItem ltr"):
                    date_span = li.find("span", class_="newspaperArticle")
                    if date_span:
                        text = date_span.get_text()
                        if re.search(r'\bA1\b', text) or re.search(r'\b1\b', text):
                            title_div = li.find("div", class_="truncatedResultsTitle")
                            if title_div:
                                title = title_div.get_text(strip=True)
                                titles_with_a1.append(title)

                titles_row = {"Date": formatted_date}
                for i, title in enumerate(titles_with_a1):
                    titles_row[f"Title {i+1}"] = title

                new_row_df = pd.DataFrame([titles_row])
                df = pd.concat([df, new_row_df], ignore_index=True)

                ############################################
                time.sleep(1.5)

                try:
                    full_text_link = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "Full text - PDF"))
                    )
                    full_text_link.click()
                except TimeoutException:
                    print(f"'Full text - PDF' button not found for {day_text}, skipping...")
                    continue
                if check_document_unavailable():
                    print("Document unavailable, navigating back and skipping this newspaper.")
                    navigate_back_to_issue()

                    continue

                driver.implicitly_wait(25)
                full_page = wait.until(EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "Page view - PDF")))
                full_page.click()
                driver.implicitly_wait(10)
                first_page_click = driver.find_element(by=By.ID, value="pdfField")
                first_page_click.clear()
                first_page_click.send_keys("1")

                go_button = driver.find_element(by=By.ID, value="submit_3")
                go_button.click()
                driver.implicitly_wait(50)
                pdf_url = get_pdf_url()
                response = requests.get(pdf_url)
                local_file_path = os.path.expanduser(f'~/Desktop/Lamis/Database/{formatted_date}.pdf')
                with open(local_file_path, 'wb') as pdf_file:
                    pdf_file.write(response.content)

                navigate_back_to_issue()

        # Save the DataFrame to a CSV file
        output_file = os.path.join(output_directory, f'{year_value}_{month_value:02}.csv')
        df.to_csv(output_file, index=False)
        print(f'Saved DataFrame for {year_value}-{month_value:02} to {output_file}')

        del df